In [8]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from distrito_federal_setor import setores
import re
import numpy as np
from lxml import html

# Configurar o Selenium com Chrome e WebDriver Manager
chrome_options = Options()
chrome_options.add_argument("--headless")  # Executar em modo headless (sem abrir o navegador)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

lista_de_imoveis = []

for pagina in range(1, 2):
    driver.get(f'https://www.dfimoveis.com.br/lancamento/df/todos/imoveis?pagina={pagina}')
    #time.sleep(5)  # Aumentar o tempo de espera para garantir que a página carregue

    # Capturar os links dos imóveis
    imoveis = driver.find_elements(By.CSS_SELECTOR, 'a.new-card')
    links_imoveis = [imovel.get_attribute('href') for imovel in imoveis]

    for link in links_imoveis:
        driver.get(link)
        #time.sleep(1)  # Aumentar o tempo de espera para garantir que a página carregue
        
        conteudo = driver.page_source
        tree = html.fromstring(conteudo)

        # Extração de informações adicionais dentro do anúncio usando XPath
        try:
            titulo = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[4]/div[1]/div[2]/h1/text()')[0].strip()
        except IndexError:
            titulo = 'N/A'
        
        try:
            subtitulo = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[6]/p/text()')[0].strip()
        except IndexError:
            subtitulo = 'N/A'
        
        try:
            preco = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[4]/div[2]/div[1]/h6/small/text()')[0].strip()
        except IndexError:
            preco = 'N/A'
        
        try:
            area = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[4]/div[2]/div[2]/h6/small/text()')[0].strip()
        except IndexError:
            area = 'N/A'
        
        try:
            imobiliaria = tree.xpath('/html/body/main/section/div/div[2]/div/div/div[2]/div[2]/h6/text()')[0].strip()
        except IndexError:
            imobiliaria = 'N/A'

        try:
            cidade = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[4]/div[3]/div[4]/h6/small/text()')[0].strip()
        except IndexError:
            cidade = 'N/A'

        try:
            detalhe_1 = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[5]/div[2]/div[1]/h6/text()')[0].strip()
        except IndexError:
            detalhe_1 = 'N/A'

        try:
            memorial_incorporacao = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[5]/div[2]/div[3]/h6/small/text()')[0].strip()
        except IndexError:
            memorial_incorporacao = 'N/A'

        try:
            codigo = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[5]/div[2]/div[2]/h6/small/text()')[0].strip()
        except IndexError:
            codigo = 'N/A'

        try:
            ultima_atualizacao = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[5]/div[2]/div[4]/h6/small/text()')[0].strip()
        except IndexError:
            ultima_atualizacao = 'N/A'

        try:
            fase = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[7]/div[1]/div[2]/h5/span/text()')[0].strip()
        except IndexError:
            fase = 'N/A'

        try:
            caracteristicas = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[8]/div[3]/ul')[0].text_content().strip()
        except IndexError:
            caracteristicas = 'N/A'

        # Detalhes dos quartos, suites e vagas
        try:
            detalhes_div = tree.xpath('/html/body/main/section/div/div[1]/div/div/div[4]/div[3]')[0]
            quarto = suite = vaga = 'N/A'
            for detalhe in detalhes_div.xpath('.//text()'):
                if 'quartos' in detalhe.lower():
                    quarto = detalhe.strip()
                elif 'suítes' in detalhe.lower():
                    suite = detalhe.strip()
                elif 'vagas' in detalhe.lower():
                    vaga = detalhe.strip()
        except IndexError:
            quarto = suite = vaga = 'N/A'

        lista_de_imoveis.append([titulo, subtitulo, link, preco, area, quarto, suite, vaga, imobiliaria, cidade, detalhe_1, memorial_incorporacao, codigo, ultima_atualizacao, fase, caracteristicas])

# Fechar o driver
driver.quit()

# Criar o DataFrame
df_imovel = pd.DataFrame(lista_de_imoveis, columns=['Título', 'Subtítulo', 'Link', 'Preço', 'Área', 'Quarto', 'Suite', 'Vaga', 'Imobiliária', 'Cidade', 'Detalhe 1', 'Memorial de Incorporação', 'Código', 'Última Atualização', 'Fase', 'Características'])

# # Remover duplicatas com base na coluna 'Link'
# df_imovel = df_imovel.drop_duplicates(subset='Link')

# # Substituir valores vazios por NaN na coluna 'Preço'
# df_imovel['Preço'] = df_imovel['Preço'].replace('', np.nan)

# # Substituir valores vazios por NaN na coluna 'Área'
# df_imovel['Área'] = df_imovel['Área'].replace('', np.nan)

# # Substituir valores vazios por NaN na coluna 'Quarto'
# df_imovel['Quarto'] = df_imovel['Quarto'].replace('N/A', np.nan)

# # Substituir valores vazios por NaN na coluna 'Suite'
# df_imovel['Suite'] = df_imovel['Suite'].replace('N/A', np.nan)

# # Substituir valores vazios por NaN na coluna 'Vaga'
# df_imovel['Vaga'] = df_imovel['Vaga'].replace('N/A', np.nan)

# # Converter a coluna 'Preço' para números
# df_imovel['Preço'] = df_imovel['Preço'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# # Converter a coluna 'Área' para números
# df_imovel['Área'] = df_imovel['Área'].str.replace(r'[^\d.]', '', regex=True).astype(float)

# # Converter as colunas 'Quarto', 'Suite' e 'Vaga' para números
# df_imovel['Quarto'] = df_imovel['Quarto'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
# df_imovel['Suite'] = df_imovel['Suite'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
# df_imovel['Vaga'] = df_imovel['Vaga'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)

# # Adicionar nova coluna 'M2' e calcular a divisão
# df_imovel['M2'] = df_imovel['Preço'] / df_imovel['Área']

# # Substituir os valores vazios por 0 nas colunas especificadas
# colunas_para_preencher = ['Preço', 'Área', 'Quarto', 'Suite', 'Vaga', 'M2']
# df_imovel[colunas_para_preencher] = df_imovel[colunas_para_preencher].fillna(0)

# # Função para extrair o setor da string de título
# def extrair_setor(titulo): 
#     # Extrair as palavras individuais do título
#     palavras = titulo.split()
#     palavras_upper = [palavra.upper() for palavra in palavras]
#     # Encontrar a primeira sigla que corresponde a um setor
#     for palavra in palavras_upper:
#         if palavra in setores:
#             return palavra
    
#     # Se nenhuma sigla for encontrada, retornar 'OUTRO'
#     return 'OUTRO'

# # Aplicar a função para extrair o setor e criar a nova coluna 'Setor'
# df_imovel['Setor'] = df_imovel['Título'].apply(extrair_setor)

# # Função para extrair o tipo do imóvel do link
# def extrair_tipo(link):
#     if "apartamento" in link:
#         return "Apartamento"
#     elif "casa" in link:
#         return "Casa"
#     elif "casa-condominio" in link:
#         return "Casa Condomínio"
#     elif "galpo" in link:
#         return "Galpão"
#     elif "garagem" in link:
#         return "Garagem"
#     elif "hotel-flat" in link or "flat" in link:
#         return "Flat"
#     elif "kitnet" in link:
#         return "Kitnet"
#     elif "loja" in link:
#         return "Loja"
#     elif "loteamento" in link:
#         return "Loteamento"
#     elif "predio-comercial" in link:
#         return "Prédio Comercial"
#     elif "predio" in link:
#         return "Prédio"
#     elif "sala" in link:
#         return "Sala"
#     elif "sala-conjunto" in link:
#         return "Sala Conjunto"
#     elif "sitio-chacara-fazenda" in link:
#         return "Sítio Chácara Fazenda"
#     else:
#         return "OUTROS"

# # Aplicar a função para extrair o tipo de imóvel e criar a nova coluna 'Tipo'
# df_imovel['Tipo'] = df_imovel['Link'].apply(extrair_tipo)

# Exportar o DataFrame para um arquivo Excel
df_imovel.to_excel(r'C:\Users\galva\OneDrive\Documentos\GitHub\web-scrapping-com-python\base_de_dados_excel\df_imoveis_data_base\df_imoveis_df_lancamento_05_2024.xlsx', index=False)
